<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
keras = tf.keras

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
# bkeras.backend.clear_session()

In [ ]:
train=pd.read_csv('/kaggle/input/final-train/train.csv')
for country in train.Territory.unique():
    train.loc[train.Territory==country,"target"]=train.loc[train.Territory==country,"target"].diff(1)
#     train.loc[train.Territory==country,"cases"]=train.loc[train.Territory==country,"cases"].diff(1)
train.dropna(inplace=True)
train.head()
train

In [ ]:
train.Territory.unique()

Let's Visualize a the curve of some countries

In [ ]:
[train.Territory=='Afghanistan','cases']

In [ ]:
def visualize_countries(country):
    plt.figure(figsize=(13,7))
#     plt.plot(train.loc[train.Territory==country,'Date'],train.loc[train.Territory==country,'cases'])
    plt.plot(train.loc[train.Territory==country,'Date'],train.loc[train.Territory==country,'target'])
#     plt.legend(['cases','target'])
    
    plt.xticks(rotation=80)
    plt.show()
    
visualize_countries("Afghanistan")

In [ ]:
visualize_countries('China')

In [ ]:
visualize_countries("United States of America (the)")

In [ ]:
visualize_countries('China')

In [ ]:
global_country=train.groupby("Territory").sum().reset_index()
global_country.max()

In [ ]:
plt.figure(figsize=(10,15))
for k in train.Territory.unique():
    plt.plot(train.loc[train.Territory==k,'Date'],train.loc[train.Territory==k,'target'])

plt.xticks(rotation=80)
plt.show()

In [ ]:
global_country['target'].describe()

In [ ]:
global_country['target'].median()

In [ ]:
train.loc[train.Territory=='China'].shape

In [ ]:
enc=LabelEncoder()
enc.fit(list(train.Territory.unique()))
enc.transform(['China'])

In [ ]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=10):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
#     dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

def preprocessing(country):
    train_country=train.loc[train.Territory==country,'target'].values
    scaler=MinMaxScaler(feature_range=(0.,1))
    train_country=scaler.fit_transform(train_country.reshape(-1,1)).reshape(1,-1)[0]
       
    #splitting training data 2/3 for the training part and 1/3 of the part 
    split_data=int(2*train_country.shape[0]/3)
    series_train=train_country[:split_data]
    series_validation=train_country[split_data:]
    print('the length for the specific dataset ',len(train_country))    
    print('the length of training set is ',series_train.shape,'and the lenght of the validation set is ',series_validation.shape)
    return [ series_train,series_validation,scaler]

def define_model(k=100):
    
    model = keras.models.Sequential([
      keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                          input_shape=[None]),
      keras.layers.LSTM(k, return_sequences=True,
                         input_shape=[None, 1]),
      keras.layers.LSTM(k, ),
#       keras.layers.Dense(100),
    #       keras.layers.SimpleRNN(1),
    #       keras.layers.SimpleRNN(100, return_sequences=True),
    #       keras.layers.SimpleRNN(100),
            keras.layers.Dense(1),
#           keras.layers.Lambda(lambda x: x * 500.0)
        ])
    return model
def define_deeper_model(num_country):
    inputs_country=tf.keras.layers.Input(shape=(1,))
    inputs_sequentials=tf.keras.layers.Input(shape=(1,))
    sequential_model=define_model()(inputs_sequentials)
    country_layers=tf.keras.layers.Embedding(num_country,32,embeddings_initializer=tf.keras.initializers.GlorotNormal)(inputs_country)
    add=tf.keras.layers.concatenate([country_layers,sequential_model])
    dens=tf.keras.layers.Dense(10)(add)
    output=tf.keras.layers.Dense(1)
    return tf.keras.Models(inputs=[inputs_sequential,inputs_coutry],outputs=output)
    

def sequential_window_dataset(series, window_size,forward_step=7):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(  window_size + forward_step, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size + forward_step))
    ds = ds.map(lambda window: (window[:-forward_step], window[window_size:]))
    return ds

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

def preprocessing_all_data(window_size,shuffle_buffer,batch_size=100,use_all=False):
    enc=LabelEncoder()
    enc.fit(train.Territory.unique())
    if use_all:
        split=train.shape[0] # no validation data 
    else:
        split=int(2*(train.loc[train.Territory=='China','target'].shape[0])/3)
        
    series=train.loc[train.Territory=='China','target'].values
    toscale=train.target.values
    scaler=MinMaxScaler(feature_range=(0,1))
    
    scaler.fit(toscale.reshape(-1,1))
    series=scaler.transform(series.reshape(-1,1)).reshape(1,-1)[0]
    
    dataset_train=window_dataset(series[:split],window_size,enc.transform(['China'])[0])
    dataset_validation=window_dataset(series[split:],window_size,enc.transform(['China'])[0])
    
    for country in train.Territory.unique():
        series=train.loc[train.Territory==country,'target'].values
        series=scaler.transform(series.reshape(-1,1)).reshape(1,-1)[0]
        #create a datset for one country
        dataset_train_country=window_dataset(series[:split],window_size,enc.transform([country])[0])
        dataset_validation_country=window_dataset(series[split:],window_size,enc.transform([country])[0])
        #concatenate in the stack
        dataset_train=dataset_train.concatenate(dataset_train_country)
        data_validation=dataset_validation.concatenate(dataset_validation_country)
        
    dataset_train=dataset_train.batch(batch_size).prefetch(100)
    dataset_validation=dataset_validation.batch(batch_size).prefetch(100)
    return [dataset_train,dataset_validation,scaler]

        
        

    

In [ ]:
#differents type of model
def define_model_Rnn():
    model = keras.models.Sequential([
      keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                          input_shape=[None]),
        
          keras.layers.SimpleRNN(100, return_sequences=True),
          keras.layers.SimpleRNN(100),
            keras.layers.Dense(1),
#           keras.layers.Lambda(lambda x: x * 500.0)
        ])
    return model


    

In [ ]:
window_size=25
dataset_train,dataset_validation,scaler_all_data=preprocessing_all_data(window_size,10,use_all=True)
#run the code below only once
train_copy=train.copy()
train.target=scaler_all_data.transform(train.target.values.reshape(-1,1)).reshape(1,-1)[0]


In [ ]:
for k in dataset_validation:
    print('first',len(k))
    for i in  k:
        print(len(i))

In [ ]:
train.target.max(),train.target.min()

In [ ]:
# window_size = 5
# train_set = window_dataset(series_train, window_size ,batch_size=10)

# valid_set = window_dataset(series_validation, window_size ,batch_size=10)
def decreasing(epochs):
    lr=1e-5
    if epochs <10:
        lr
    else:
        lr=lr*(1**(-epochs/10))
    return lr
result={}
# for k in [20,50,100]:
#     model = define_model(k)

#     # optimizer = keras.optimizers.SGD(lr=1.5e-6, momentum=0.9)
#     optimizer=keras.optimizers.Nadam(learning_rate=1e-5)
#     model.compile(loss=keras.losses.Huber(),
#                   optimizer=optimizer,
#                   metrics=["mae"])


#     early_stopping = keras.callbacks.EarlyStopping(patience=50)
#     lr_schedule_decreasing = keras.callbacks.LearningRateScheduler(decreasing)
#     model_checkpoint = keras.callbacks.ModelCheckpoint(
#         "my_checkpoint", save_best_only=True)



#     result[k]=model.fit(dataset_train, epochs=50,
#               validation_data=dataset_validation,
#               callbacks=[early_stopping, model_checkpoint,lr_schedule_decreasing])


In [ ]:
# result={"10": result[0],"50":result[1],"100": result[2]}
result

In [ ]:
# def plot_history(result,key):

#     plt.boxplot([result[k].history[key] for k in result.keys()],showmeans=True)
# #     ax.set_title=f'History of {key}'
#     plt.show()
    
# #         plt.xticks=[10,50,100]
# #     plt.plot(history.history['loss'],label=label)

# plot_history(result,'loss')
# plot_history(result,'val_loss')


    
#     result[0].history.loss

the model with 100 consecutive cellule is better

In [ ]:
# Optimise learning rate
# window_size = 5
# train_set = window_dataset(series_train, window_size)
# model=define_deeper_model(len(train.Territory.unique()))
model=define_model(100)
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 30))
reset_states = ResetStatesCallback()
# optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
optimizer=keras.optimizers.Nadam(learning_rate=1e-8)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(dataset_train, epochs=500,
                    callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])

In [ ]:
model = define_model(100)

# optimizer = keras.optimizers.SGD(lr=1.5e-6, momentum=0.9)
optimizer=keras.optimizers.Nadam(learning_rate=1e-6)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])


# early_stopping = keras.callbacks.EarlyStopping(patience=20)
lr_schedule_decreasing = keras.callbacks.LearningRateScheduler(decreasing)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint", save_best_only=True)
model.fit(dataset_train, epochs=150,
#           validation_data=dataset_validation,
          callbacks=[
#               early_stopping,
              model_checkpoint,lr_schedule_decreasing])


In [ ]:
model = keras.models.load_model("my_checkpoint")

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast


In [ ]:
# plot result
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)
    


In [ ]:
#lets visualize a long term prediction
#let considere china
def forecast_forward(step_forward,country,train):
    """
    forecast for a specific country a setp_forward time prediction 
    -step_forward: number of succesive prediction that have to be done
    - country : name of the country in the train dataset
    """

    country_series=train.loc[train.Territory==country,"target"].values
    predict=country_series
    for k in range(step_forward):

        predict=np.append(predict,model_forecast(model,predict,window_size)[-1])
    print(f'the predicted series has a size of {len(predict)}')
    
    return predict 

def visualize_forecast(country,step_forward=7,window_size=window_size):
    """
    it create graph of prediction and actual graph
    
    """
    china_series=train.loc[train.Territory==country,"target"].values
    to_predict=china_series[-step_forward:]
    past_series=china_series[:-step_forward]
    predict=past_series
    for k in range(step_forward):
    #     print(k,predict)
#         print(len(model_forecast(model,predict,window_size)))
        predict=np.append(predict,model_forecast(model,predict,window_size)[-1])

    plt.plot(range(len(predict)),predict,'*')
#     plt.plot(china_series,'o')
    plt.plot(past_series)



In [ ]:
forecast_forward(6,'China',train)

In [ ]:
original_train=pd.read_csv('/kaggle/input/final-train/train.csv') 
# dont forget to have more element than your input window lenght
new_train=train.loc[pd.to_datetime(train.Date)>=pd.to_datetime('2020-03-06')]
step_forward=(pd.to_datetime('2020-06-07')-pd.to_datetime(train.Date, format='%m/%d/%y').max()).days
step_forward

In [ ]:
new_train.shape[0]/train.Territory.nunique()+59

In [ ]:
train.Date.max()

In [ ]:
train

In [ ]:
19646/len(train.Territory.unique())

In [ ]:
import random
def cumulative_op(array,first_element):
    for k in range(len(array)):
        array[k]=array[k]+first_element
        first_element=array[k]
    return array

def submit(scaler):
    original_train=pd.read_csv('/kaggle/input/final-train/train.csv')
    
    # dont forget to have more element than your input window lenght
    new_train=train.loc[pd.to_datetime(train.Date)>=pd.to_datetime('2020-03-06')]
    step_forward=(pd.to_datetime('2020-06-07')-pd.to_datetime(train.Date, format='%m/%d/%y').max()).days
    dates = pd.date_range(start='2020-03-06', end='2020-06-07', freq='1d')
    ids = []
    prediction=np.array([])
    for country in sorted(train['Territory'].unique()):
        for d in dates:
            ids.append(country + ' X ' + d.strftime('%-m/%-d/%y'))
        prediction=np.append(prediction, cumulative_op( scaler.inverse_transform( forecast_forward(step_forward,country,new_train).reshape(-1,1)).reshape(1,-1)[0],original_train.loc[(pd.to_datetime(original_train.Date)==pd.to_datetime('2020-03-06')) &(original_train.Territory==country) ,'target'].values[0]   ))
        
        
    print(len(prediction),len(ids))
    ss = pd.DataFrame({
        'Territory X Date':ids,
        'target': prediction
    })
    
#     for k in range(ss.shape[0]):
#         ss['target'].iloc[k]=random.randint(0,1000)
    ss.loc[ss.target<=0,'target']=0
    ss.target=ss.target.apply(lambda x:round(x))
    ss.to_csv('SampleSubmission_mine.csv', index=False)
    print(ss['target'].sum())
    ss.head()
submit(scaler_all_data)

In [ ]:
train.target.apply(lambda x : round(x) )

In [ ]:
# TODO list make a custom model that take in addition a tabular layer for embedding country and the max infected number of people. 
#it will help the model distinguish between country that have less infection .